## To get all the components that affected and fixed

In [1]:
import requests

def fetch_cves(limit=10000):
    base_url = "https://access.redhat.com/labs/securitydataapi/cve.json"
    cves = []
    page = 1
    per_page = 100  # Number of CVEs per page (max allowed by the API)

    while len(cves) < limit:
        url = f"{base_url}?page={page}&per_page={per_page}"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break
        
        data = response.json()
        if not data:
            break  # No more data available
        
        cves.extend(data)
        page += 1

        print(f"Fetched {len(cves)} CVEs so far...")

        if len(data) < per_page:
            break  # No more pages available

    return cves[:limit]

# Fetch the first 20,000 CVEs
cves = fetch_cves(limit=20000)

# Print the total number of CVEs fetched
print(f"Total CVEs fetched: {len(cves)}")


Fetched 100 CVEs so far...
Fetched 200 CVEs so far...
Fetched 300 CVEs so far...
Fetched 400 CVEs so far...
Fetched 500 CVEs so far...
Fetched 600 CVEs so far...
Fetched 700 CVEs so far...
Fetched 800 CVEs so far...
Fetched 900 CVEs so far...
Fetched 1000 CVEs so far...
Fetched 1100 CVEs so far...
Fetched 1200 CVEs so far...
Fetched 1300 CVEs so far...
Fetched 1400 CVEs so far...
Fetched 1500 CVEs so far...
Fetched 1600 CVEs so far...
Fetched 1700 CVEs so far...
Fetched 1800 CVEs so far...
Fetched 1900 CVEs so far...
Fetched 2000 CVEs so far...
Fetched 2100 CVEs so far...
Fetched 2200 CVEs so far...
Fetched 2300 CVEs so far...
Fetched 2400 CVEs so far...
Fetched 2500 CVEs so far...
Fetched 2600 CVEs so far...
Fetched 2700 CVEs so far...
Fetched 2800 CVEs so far...
Fetched 2900 CVEs so far...
Fetched 3000 CVEs so far...
Fetched 3100 CVEs so far...
Fetched 3200 CVEs so far...
Fetched 3300 CVEs so far...
Fetched 3400 CVEs so far...
Fetched 3500 CVEs so far...
Fetched 3600 CVEs so far...
F

In [2]:
cve_resources=[]
for cve in cves:
    cve_resources.append(cve['resource_url'])

In [3]:
len(cve_resources)

20000

In [4]:
rhsas = []
cve_names = []
package_names = []
fix_states = []

In [5]:
cve_names = []
package_names = []
fix_states = []
rhsas = []

for cve_resource in cve_resources:
    cve_response = requests.get(cve_resource)
    if cve_response.status_code != 200:
        continue  # Skip if the request fails

    security_data = cve_response.json()
    if 'affected_release' not in security_data:
        continue  # Skip if there are no affected releases

    for affected_release in security_data['affected_release']:
        package = affected_release.get('package', '')
        if "openshift4/" in package:
            cve_names.append(security_data['name'])
            package_names.append(package)
            fix_states.append('Fixed - Curated Data')
            rhsas.append(affected_release.get('advisory', ''))

In [6]:
len(rhsas)

30556

In [7]:
from pandas import DataFrame
df = DataFrame({'CVE': cve_names, 'package': package_names, 'Fix State': fix_states, 'Advisory' : rhsas})
df.to_excel('affected_fixed.xlsx', sheet_name='sheet1', index=False)